# **1D Simulator**
This code is used to simulate 1D equations in order to create some data for ai training. 

The first part will be the reproduction burger equation in 1D such as :
$$
\begin{equation}
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
\end{equation}
$$

In [1]:
import os
import random 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import ndimage
import collections.abc
import matplotlib.pyplot as plt

# Neural Network

The idea behind this code is to resolve the learning approach from the model. Going from getting the Burger formula to use the Laplace approach in order to the model to learn about the equation values.

In [2]:
class LearnableDerivative(nn.Module):
    def __init__(self, kernel_size=5, channels=1, learn_scaling=True):
        super().__init__()
        self.kernel_size = kernel_size
        self.learn_scaling = learn_scaling
        
        # Convolution to learn spatial derivatives
        self.conv = nn.Conv1d(channels, channels, kernel_size, 
                             padding=kernel_size//2, bias=False, padding_mode='replicate')
        
        # Learnable scaling factor (replaces 1/dx²)
        if learn_scaling:
            self.scale = nn.Parameter(torch.tensor(1.0))
        else:
            self.scale = 1.0
            
        # Initialize as approximation of Laplacian
        self.initialize_as_laplacian(kernel_size)
        
    def initialize_as_laplacian(self, kernel_size):
        """Initialize convolution weights to approximate Laplacian"""
        weight = torch.zeros(1, 1, kernel_size)
        center = kernel_size // 2
        
        if kernel_size == 3:
            # Standard 3-point stencil: [1, -2, 1]
            weight[0, 0, center-1] = 1.0
            weight[0, 0, center] = -2.0
            weight[0, 0, center+1] = 1.0
        elif kernel_size == 5:
            # Higher order approximation: [-1/12, 4/3, -5/2, 4/3, -1/12]
            weight[0, 0, center-2] = -1/12
            weight[0, 0, center-1] = 4/3
            weight[0, 0, center] = -5/2
            weight[0, 0, center+1] = 4/3
            weight[0, 0, center+2] = -1/12
        elif kernel_size == 7:
            # Even higher order
            weight[0, 0, center-3] = 1/90
            weight[0, 0, center-2] = -3/20
            weight[0, 0, center-1] = 3/2
            weight[0, 0, center] = -49/18
            weight[0, 0, center+1] = 3/2
            weight[0, 0, center+2] = -3/20
            weight[0, 0, center+3] = 1/90
            
        self.conv.weight = nn.Parameter(weight)
        self.conv.weight.requires_grad = True
        
    def forward(self, u):
        # u shape: (batch, channels, spatial)
        result = self.conv(u)
        if self.learn_scaling:
            result = result * self.scale
        return result

    def get_effective_weights(self):
        """Get the effective finite difference weights"""
        with torch.no_grad():
            weights = self.conv.weight.data.clone()
            if self.learn_scaling:
                weights = weights * self.scale.item()
            return weights.squeeze().numpy()

In [3]:
# Main change regarding the burgers1D learning with Laplace.Before -> Burgers1D review Samuel's code to confirm the accuracy.

class Burgers1DLearned:
    def __init__(self, grid, nu, initial_condition, 
                 boundary_condition=None, cfl_safety=0.5,
                 learn_laplacian=True, kernel_size=5, device='cpu'):
        self.grid = grid
        self.nu = float(nu)
        self.initial_condition = initial_condition
        self.boundary_condition = boundary_condition or bc_periodic
        self.cfl_safety = float(cfl_safety)
        self.device = device
        
        # Learnable components
        self.learn_laplacian = learn_laplacian
        if learn_laplacian:
            self.laplacian_net = LearnableDerivative(kernel_size=kernel_size).to(device)
            self.optimizer = optim.Adam(self.laplacian_net.parameters(), lr=1e-3)
            self.loss_history = []
        else:
            self.laplacian_net = None

    def compute_laplacian(self, U):
        """Compute Laplacian using either learned or analytical method"""
        if self.learn_laplacian and self.laplacian_net is not None:
            return self.learned_laplacian(U)
        else:
            return self.analytical_laplacian(U)
    
    def learned_laplacian(self, U):
        """Compute Laplacian using learned convolution"""
        # Convert to tensor and add batch/channel dimensions
        u_tensor = torch.FloatTensor(U).unsqueeze(0).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            laplacian_tensor = self.laplacian_net(u_tensor)
        
        # Convert back to numpy and remove extra dimensions
        laplacian = laplacian_tensor.squeeze().cpu().numpy()
        
        # Apply the correct scaling (1/dx²)
        scaling_factor = 1.0 / (self.grid.dx ** 2)
        return laplacian * scaling_factor
    
    def analytical_laplacian(self, U):
        """Original finite difference Laplacian"""
        #return Burgers1D(self.grid, self.nu, self.initial_condition).analytical_laplacian(U) OLD 
        return Burgers1D(self.grid, self.nu, self.initial_condition).deriv2order(U)

    def generate_high_res_reference(self, U_low_res, scale_factor=2):
        """Generate high-resolution reference using interpolation"""
        # Use spline interpolation for smooth high-res data
        from scipy import ndimage
        
        # Interpolate to higher resolution
        U_high_res = ndimage.zoom(U_low_res, scale_factor, order=3)
        return U_high_res

    def compute_reference_laplacian(self, U, dx):
        """Compute reference Laplacian using high-order finite differences"""
        # Use 5-point stencil for accurate reference
        laplacian = np.zeros_like(U)
        n = len(U)
        
        if n >= 5:
            # Interior points (5-point stencil)
            laplacian[2:-2] = (-U[4:] + 16*U[3:-1] - 30*U[2:-2] + 16*U[1:-3] - U[:-4]) / (12 * dx**2)
            # Boundaries with lower order
            laplacian[0] = (U[1] - 2*U[0] + U[0]) / dx**2
            laplacian[1] = (U[2] - 2*U[1] + U[0]) / dx**2
            laplacian[-2] = (U[-1] - 2*U[-2] + U[-3]) / dx**2
            laplacian[-1] = (U[-1] - 2*U[-1] + U[-2]) / dx**2
        
        return laplacian

    def train_laplacian(self, training_data=None, epochs=1000, batch_size=8):
        """Train the Laplacian network on accurate reference data"""
        if not self.learn_laplacian:
            return
            
        if training_data is None:
            # Generate training data if not provided
            training_data = self.generate_training_data(n_samples=100)
        
        criterion = nn.MSELoss()
        self.loss_history = []
        
        for epoch in range(epochs):
            total_loss = 0
            num_batches = 0
            
            # Shuffle training data
            np.random.shuffle(training_data)
            
            for i in range(0, len(training_data), batch_size):
                batch_data = training_data[i:i+batch_size]
                batch_loss = 0
                
                for U_low_res in batch_data:
                    # Generate high-resolution reference
                    U_high_res = self.generate_high_res_reference(U_low_res)
                    dx_high_res = self.grid.dx / 2  # Higher resolution
                    
                    # Compute target Laplacian from high-res data using accurate method
                    target_laplacian = self.compute_reference_laplacian(U_high_res, dx_high_res)
                    
                    # Downsample target to match low-res grid
                    target_low_res = target_laplacian[::2]
                    
                    # Prepare tensors
                    U_low_tensor = torch.FloatTensor(U_low_res).unsqueeze(0).unsqueeze(0).to(self.device)
                    target_tensor = torch.FloatTensor(target_low_res).unsqueeze(0).unsqueeze(0).to(self.device)
                    
                    # Forward pass
                    self.optimizer.zero_grad()
                    pred_laplacian = self.laplacian_net(U_low_tensor)
                    
                    # Compute loss (with proper scaling)
                    scaling = 1.0 / (self.grid.dx ** 2)
                    pred_laplacian_scaled = pred_laplacian * scaling
                    
                    loss = criterion(pred_laplacian_scaled, target_tensor)
                    
                    # Backward pass
                    loss.backward()
                    self.optimizer.step()
                    
                    batch_loss += loss.item()
                
                total_loss += batch_loss / len(batch_data)
                num_batches += 1
            
            avg_loss = total_loss / num_batches if num_batches > 0 else total_loss
            self.loss_history.append(avg_loss)
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {avg_loss:.6f}")
                # Print learned weights for monitoring
                if self.learn_laplacian:
                    weights = self.laplacian_net.get_effective_weights()
                    print(f"  Learned weights: {weights}")

    def generate_training_data(self, n_samples=100):
        """Generate diverse training data with accurate Laplacians"""
        training_data = []
        
        # Create various initial conditions
        speeds = np.random.uniform(1.0, 4.0, n_samples)
        kinds = ["shock", "rarefaction", "sine", "smooth"]
        
        for i in range(n_samples):
            speed = speeds[i]
            kind = kinds[i % len(kinds)]
            
            # Generate initial condition - FIXED LINE
            ic_fn, _ = make_initial_condition_burgers_fn(self.grid.x, speed, kind)
            u0 = ic_fn(self.grid.x)
            
            # Add some noise or variations
            if np.random.random() > 0.5:
                noise = np.random.normal(0, 0.1, len(u0))
                u0 = u0 + noise
            
            training_data.append(u0)
        
        return training_data

    def simulate(self):
        """Run simulation using learned operators"""
        # Use the parent class simulation but with learned Laplacian
        parent_sim = Burgers1D(self.grid, self.nu, self.initial_condition,
                              self.boundary_condition, self.cfl_safety)
        
        nt = len(self.grid.t)
        nx = len(self.grid.x)
        dx = float(self.grid.dx)
        dt = float(self.grid.dt)
        nu = float(self.nu)

        U = np.zeros((nt, nx))
        U[0, :] = self.initial_condition(self.grid.x)

        parent_sim.check_cfl_burgers(U[0, :])

        for n in range(nt - 1):
            u = U[n, :]

            up = parent_sim._apply_boundary(u)
            uL = up[:-1]
            uR = up[1:]
            Fh = parent_sim.rusanov_flux(uL, uR)
            conv = -(Fh[1:] - Fh[:-1]) / dx

            # Use learned Laplacian
            diff = nu * self.compute_laplacian(u)

            U[n + 1, :] = u + dt * (conv + diff)

            parent_sim.check_cfl_burgers(U[n + 1, :])

        return U

# Physical Simulator

## 1D Grid creation helper

In [4]:
class Grid1D:
    def __init__(self, nb_points_x, x_min, x_max, t_max, dt):
        if x_max < x_min:
            x_min, x_max = x_max, x_min
        self.nb_points_x = int(nb_points_x)
        self.x_min, self.x_max = float(x_min), float(x_max)
        self.dt = float(dt)

        # temps
        self.nb_points_t = int(np.floor(t_max / dt)) + 1
        self.t_max = self.dt * (self.nb_points_t - 1)

        # espace
        self.dx = (self.x_max - self.x_min) / (self.nb_points_x - 1)
        self.x = np.linspace(self.x_min, self.x_max, self.nb_points_x)
        self.t = np.linspace(0, self.t_max, self.nb_points_t)

    def save_npz(self, path, U, *, nu=None, speed=None, tag=""):
        meta = dict(
            U=U.astype(np.float32),
            x=self.x.astype(np.float32),
            t=self.t.astype(np.float32),
            dx=np.float32(self.dx),
            dt=np.float32(self.dt),
        )
        if nu is not None:
            meta["nu"] = np.float32(nu)
        if speed is not None:
            meta["speed"] = np.float32(speed)
        if tag:
            meta["tag"] = np.str_(tag)
        folder = os.path.dirname(path)
        if folder:
            os.makedirs(folder, exist_ok=True)
        np.savez_compressed(path, **meta)

    # (Optionnel) petites aides pour viz rapide
    def plot(self, U, title="Burgers 1D", xlabel="X", ylabel="T", label=None, cmap='seismic'):
        plt.figure(figsize=(10, 6))
        plt.imshow(
            U,
            extent=[self.x.min(), self.x.max(), self.t.max(), self.t.min()],
            cmap=cmap, interpolation='nearest', aspect='auto', origin='upper'
        )
        plt.colorbar(label=label)
        plt.title(title); plt.xlabel(xlabel); plt.ylabel(ylabel)
        plt.show()

    def plot_initial_condition(self, U, title="U(x, t=0)", label=None, cmap='seismic'):
        # accepte U 2D (t,x) ou 1D (x)
        U0 = U[0, :] if getattr(U, "ndim", 1) == 2 else U
        plt.figure(figsize=(10, 3))
        plt.plot(self.x, U0, linewidth=1.8)
        if label:
            plt.ylabel(label)
        plt.title(title)
        plt.xlabel("X")
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.tight_layout()
        plt.show()

## Initial conditions

This section documents the simple 1D **initial-condition generators** used by the Burgers simulator.  
Each type of initial condition defines the starting velocity field \( u(x, 0) \) over the spatial domain.

- `shock`: step discontinuity (left/right states of opposite sign)  
- `shock_with_gap`: shock separated by a zero plateau in the center  
- `rarefaction`: opposite of shock — a smooth fan separating two constant states  
- `sine`: random sinusoidal perturbation  
- `smooth`: random smooth profile (low-pass filtered noise)

### Mathematical intent

Typical analytic shapes implemented:

- **Shock** (discontinuous jump):  
  \[
  u(x,0)=
  \begin{cases}
  +U, & x < 0 \\
  -U, & x \ge 0
  \end{cases}
  \]

- **Rarefaction** (reverse sign):  
  \[
  u(x,0)=
  \begin{cases}
  -U, & x < 0 \\
  +U, & x \ge 0
  \end{cases}
  \]

- **Sinusoidal**:  
  \[
  u(x,0) = U\,\sin\!\left(\frac{2\pi kx}{x_{\max}} + \phi\right),
  \]
  where \(k\) is a random integer mode (1–4) and \(\phi\) a random phase.

- **Smooth**: random low-frequency fluctuations obtained by averaging local noise.

The helper `make_initial_condition_burgers_fn()` wraps these generators and returns a callable function \(\lambda(x)\) producing the sampled field, together with the chosen kind label.


In [5]:
def sample_initial_condition_burgers(x, speed, kind="shock"):
    if kind == "shock":
        u0 = np.where(x < 0, speed, -speed)
    elif kind == "shock_with_gap":
        u0 = np.where((x >= -1.0) & (x <= 1.0), 0.0, np.where(x < 0, 1.0, -1.0)) * speed
    elif kind == "rarefaction":
        u0 = np.where(x < 0, -speed, speed)
    elif kind == "sine":
        k = np.random.randint(1, 5)
        phase = np.random.uniform(0, 2*np.pi)
        xmax = max(1e-12, x.max())
        u0 = np.sin(2*np.pi*k*x/xmax + phase) * speed
    else:  # "smooth" aléatoire
        u0 = (np.random.rand(len(x)) * 2 - 1) * speed
        u0 = (u0 + np.roll(u0, 1) + np.roll(u0, -1)) / 3.0
    return u0

def make_initial_condition_burgers_fn(x, speed, kind=None):
    if kind is None:
        kind = np.random.choice(["shock", "rarefaction", "sine", "smooth"])
    u0 = sample_initial_condition_burgers(x, speed, kind)
    return (lambda _x: u0), kind

## Boundary conditions

This section documents the simple 1D boundary-condition helpers used by the simulator:

- `bc_dirichlet_clamp`: Dirichlet (clamped) boundary values
- `bc_periodic`: periodic / wrap-around
- `bc_neumann_zero`: zero-gradient (Neumann) — mirrored ghost cells

### Mathematical intent

- Dirichlet (clamp): fix the solution at the two domain ends, e.g.

  $$u(x_{\min},t)=u_{\text{left}},\qquad u(x_{\max},t)=u_{\text{right}}.$$

- Periodic: values wrap around the domain, i.e. $$u(x+L)=u(x)\quad(L=x_{\max}-x_{\min}).$$

- Neumann zero-gradient: enforce $$\partial_x u|_{\text{boundary}} = 0$$ which is commonly implemented by mirroring the edge values into ghost cells.

In [6]:
# Dirichlet: clamp to fixed endpoints (values stay equal to current ends)
def bc_dirichlet_clamp(u):
    return (u[0], u[-1])  # returns tuple -> class pads with these

# Periodic (wrap)
def bc_periodic(u):
    up = np.empty(u.size + 2, dtype=u.dtype)
    up[1:-1] = u
    up[0]  = u[-1]
    up[-1] = u[0]
    return up  # already padded

# Neumann zero-gradient (mirror)
def bc_neumann_zero(u):
    up = np.empty(u.size + 2, dtype=u.dtype)
    up[1:-1] = u
    up[0]  = u[0]
    up[-1] = u[-1]
    return up


## Numerical scheme for the 1D Burgers equation

---

### 1. Continuous form

The continuous Burgers equation is:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
$$

---

### 2. Discretization overview

We consider a **uniform grid** with spacing $\Delta x$ and a **time step** $\Delta t$.

The semi-discrete conservative form reads:
$$
\frac{d u_i}{d t} = -\frac{F_{i+1/2}-F_{i-1/2}}{\Delta x}
+ \nu \frac{u_{i+1}-2u_i+u_{i-1}}{\Delta x^2}.
$$

Here, $F(u)=u^2/2$ is the **numerical flux**, and $F_{i+1/2}$ denotes its interface approximation.

---

### 3. Convective flux choices

#### (a) Central flux (second-order, non-dissipative)
$$
F_{i+1/2} = \frac{1}{2}\left(\frac{u_{i+1}^2}{2} + \frac{u_i^2}{2}\right)
$$
⚠️ Not stable for strong gradients or shocks.

#### (b) Upwind / Godunov flux (shock-stable)

For the Rusanov (local Lax–Friedrichs) flux:
$$
F_{i+1/2} =
\frac{1}{2}\left(\frac{u_i^2}{2} + \frac{u_{i+1}^2}{2}\right)
- \frac{1}{2}\lambda_{i+1/2}(u_{i+1}-u_i),
$$
where
$$
\lambda_{i+1/2} = \max(|u_i|,|u_{i+1}|)
$$
acts as a **numerical dissipation coefficient**.

A simple upwind version uses the **sign of $u_i$**:
- If $u_i > 0$: use backward difference.  
- If $u_i < 0$: use forward difference.  
This adds dissipation and stabilizes shocks.

---

### 4. Diffusive term

A second-order centered approximation:
$$
\partial_{xx} u(x_i,t^n) \approx
\frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{\Delta x^2}.
$$

---

### 5. Time discretization (explicit Euler)

Combining both contributions, the explicit update reads:
$$
u_i^{n+1} = u_i^n
- \frac{\Delta t}{\Delta x}\big(F_{i+1/2}^n - F_{i-1/2}^n\big)
+ \nu \Delta t\;\frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{\Delta x^2}.
$$

This gives the **conservative discrete form**:  
flux differences for convection + centered second difference for diffusion.

---

### 6. Stability / CFL constraints

#### (a) Convective CFL (hyperbolic term)
$$
\Delta t \le \frac{\Delta x}{\max_i |u_i|}
$$

#### (b) Diffusive constraint (parabolic term)
$$
\Delta t \le \frac{\Delta x^2}{2\nu}
$$

#### (c) Practical combined condition
$$
\Delta t \le
\min\!\left(
  C_c\frac{\Delta x}{\max|u|},
  C_d\frac{\Delta x^2}{2\nu}
\right)
$$
with safety factors $C_c, C_d \in (0,1)$ (typically 0.5).  
If violated, the explicit scheme will diverge or produce non-physical oscillations.

---


In [7]:
def generate_training_data_for_learning(n_samples=100, grid_params=None):
    """Generate high-resolution data for training the learned operators"""
    if grid_params is None:
        grid_params = {'nbx': 256, 'x_min': -5, 'x_max': 5, 'dt': 1e-3, 'n_steps': 50}
    
    training_data = []
    
    # Create high-resolution grid
    high_res_grid = make_grid(**grid_params)
    
    for i in range(n_samples):
        # Generate high-resolution simulation
        speed = np.random.uniform(1.0, 4.0)
        ic_fn, _ = make_initial_condition_burgers(high_res_grid.x, speed, kind=None)
        
        sim = Burgers1D(high_res_grid, nu=0.1, initial_condition=ic_fn, 
                       boundary_condition=bc_neumann_zero)
        U_high_res = sim.simulate()
        
        # Use the final state as training sample
        training_data.append(U_high_res[-1, :])
    
    return training_data

In [8]:
class Burgers1D:
    def __init__(self, grid, nu, initial_condition, boundary_condition=bc_periodic, cfl_safety=0.5):
        self.grid = grid
        self.nu = float(nu)
        self.initial_condition = initial_condition
        self.boundary_condition = boundary_condition
        self.cfl_safety = float(cfl_safety)

    @staticmethod
    def rusanov_flux(uL, uR):
        lam = np.maximum(np.abs(uL), np.abs(uR))
        return 0.5 * (0.5*uL*uL + 0.5*uR*uR) - 0.5 * lam * (uR - uL)

    def deriv2order(self, U):
        nx = U.shape[0]
        d2 = np.zeros_like(U)
        if nx >= 3:
            d2[1:-1] = (U[2:] - 2*U[1:-1] + U[:-2]) / (self.grid.dx ** 2)
        if nx >= 2:
            d2[0]  = (U[1] - 2*U[0]  + U[0])   / (self.grid.dx ** 2)
            d2[-1] = (U[-1] - 2*U[-1] + U[-2]) / (self.grid.dx ** 2)
        return d2

    def _apply_boundary(self, u):
        if self.boundary_condition is None:
            return np.concatenate(([u[0]], u, [u[-1]]))
        padded = self.boundary_condition(u)
        return np.asarray(padded)

    def check_cfl_burgers(self, u_now):
        dx = abs(float(self.grid.dx))
        dt = float(self.grid.dt)
        nu = float(self.nu)
        umax = float(np.max(np.abs(u_now))) + 1e-14
        conv_limit = dx / umax
        diff_limit = dx * dx / (2.0 * nu + 1e-14)
        limit = self.cfl_safety * min(conv_limit, diff_limit)
        if dt > limit:
            raise ValueError(
                "Unstable explicit step:\n"
                f"  dt={dt:.3e} > safety*min(dx/|u|, dx^2/(2ν))={limit:.3e}\n"
                f"  (dx={dx:.3e}, ν={nu:.3e}, max|u|={umax:.3e}, safety={self.cfl_safety:.2f})"
            )

    def simulate(self):
        nt = len(self.grid.t)
        nx = len(self.grid.x)
        dx = float(self.grid.dx)
        dt = float(self.grid.dt)
        nu = float(self.nu)

        U = np.zeros((nt, nx))
        U[0, :] = self.initial_condition(self.grid.x)

        self.check_cfl_burgers(U[0, :])

        for n in range(nt - 1):
            u = U[n, :]

            up = self._apply_boundary(u)       # taille nx+2
            uL = up[:-1]
            uR = up[1:]
            Fh = self.rusanov_flux(uL, uR)     # taille nx+1
            conv = -(Fh[1:] - Fh[:-1]) / dx

            diff = nu * self.deriv2order(u)

            U[n + 1, :] = u + dt * (conv + diff)

            self.check_cfl_burgers(U[n + 1, :])

        return U

## Generation Helper

In [9]:
def _tolist(x):
    return list(x) if (isinstance(x, collections.abc.Iterable) and not isinstance(x, (str, bytes))) else [x]

def make_grid(nbx, x_min, x_max, dt, *, n_steps=None, t_final=None):
    if t_final is None:
        if n_steps is None:
            raise ValueError("Spécifie n_steps ou t_final.")
        t_final = (int(n_steps) - 1) * dt
    return Grid1D(int(nbx), float(x_min), float(x_max), float(t_final), float(dt))

def run_one_sim_burgers(grid, nu, speed, ic_kind=None, cfl_safety=0.5, max_retries=4, boundary_condition=bc_periodic):
    dt = grid.dt
    nbx = grid.nb_points_x
    x_min, x_max = grid.x_min, grid.x_max
    n_steps = len(grid.t)
    ic_fn, kind_used = make_initial_condition_burgers_fn(grid.x, speed, ic_kind)
    for k in range(max_retries):
        try:
            sim = Burgers1D(grid, nu, ic_fn, boundary_condition=boundary_condition, cfl_safety=cfl_safety)
            U = sim.simulate()
            return grid, U, kind_used
        except ValueError as e:
            if "unstable" in str(e).lower():
                dt *= 0.5
                grid = make_grid(nbx, x_min, x_max, dt, n_steps=n_steps)
            else:
                raise
        if k == max_retries - 2 and ic_kind is None:
            ic_fn, kind_used = make_initial_condition_burgers_fn(grid.x, speed, None)
    raise RuntimeError("Impossible de stabiliser la simulation après plusieurs essais.")


def generate_dataset_burgers(
    *,
    out_dir="generated_1d_burgers",
    nbx=30, x_min=-5, x_max=5, dt=5e-3, n_steps=20,
    nu=0.1, speed=4.0,
    boundary_condition=bc_periodic,
    ic_kinds=None,
    n_train=100, n_test=20,
    cfl_safety=0.5
):
    """Tous les fichiers sont stockés dans generated_1d_burgers/{train,test}."""
    train_dir = os.path.join(out_dir, "train")
    test_dir  = os.path.join(out_dir, "test")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir,  exist_ok=True)

    xmins   = _tolist(x_min)
    xmaxs   = _tolist(x_max)
    stepsLs = _tolist(n_steps)
    speeds  = _tolist(speed)

    if len(xmins) != len(xmaxs):
        if len(xmins) == 1:
            xmins = xmins * len(xmaxs)
        elif len(xmaxs) == 1:
            xmaxs = xmaxs * len(xmins)
        else:
            raise ValueError("x_min et x_max doivent avoir la même longueur ou être scalaires.")
    x_pairs = list(zip(xmins, xmaxs))

    def do_split(N, split_dir, xmin_val, xmax_val, nst_val, spd_val):
        for i in range(N):
            grid0 = make_grid(nbx, xmin_val, xmax_val, dt, n_steps=int(nst_val))
            kind = None if ic_kinds is None else random.choice(ic_kinds)
            grid_i, U, kind_used = run_one_sim_burgers(
                grid0, nu=float(nu), speed=float(spd_val),
                ic_kind=kind, cfl_safety=cfl_safety, boundary_condition=boundary_condition
            )

            tag = f"{kind_used}|x=[{xmin_val},{xmax_val}]|T={int(nst_val)}|v={float(spd_val)}"
            # 🔧 Ajoute x_min/x_max dans le nom de fichier pour éviter l'écrasement
            filename = (
                f"sample_{i:04d}"
                f"_x{float(xmin_val):+.2f}_{float(xmax_val):+.2f}"
                f"_v{float(spd_val):.2f}"
                f"_T{int(nst_val)}.npz"
            )

            grid_i.save_npz(
                os.path.join(split_dir, filename),
                U, nu=float(nu), speed=float(spd_val), tag=tag
            )


    for (xmin, xmax) in x_pairs:
        for nst in stepsLs:
            for spd in speeds:
                do_split(n_train, train_dir, xmin, xmax, nst, spd)
                do_split(n_test,  test_dir,  xmin, xmax, nst, spd)


## File visualization

In [10]:
def visualize_random_sample(out_dir="generated_1d_burgers/test", cmap="seismic", label="U", title_prefix="Burgers"):
    files = [f for f in os.listdir(out_dir) if f.endswith(".npz")]
    if not files:
        raise FileNotFoundError(f"Aucun .npz dans {out_dir}")
    sample = random.choice(files)
    path = os.path.join(out_dir, sample)
    print(f"📂 Loaded file: {sample}")

    data = np.load(path, allow_pickle=True)
    U = data["U"]; x = data["x"]; t = data["t"]; dt = float(data["dt"])
    tag = data["tag"].item() if "tag" in data else ""
    nu  = float(data["nu"]) if "nu" in data.files else None
    spd = float(data["speed"]) if "speed" in data.files else None

    grid_vis = Grid1D(len(x), float(x.min()), float(x.max()), float(t.max()), dt)
    grid_vis.plot_initial_condition(U, title=f"Initial condition — {tag or '?'}", label=label, cmap=cmap)

    tit = f"{title_prefix} (ν={nu:.3f}, speed={spd:.3f}, type={tag or '?'})"
    grid_vis.plot(U, title=tit, xlabel="X", ylabel="T", label=label, cmap=cmap)


# Data Generation

In [11]:
# This is Samuel's to review later on
'''generate_dataset_burgers(
    out_dir="generated_1d_burgers",
    nbx=128,
    x_min=[-5, -2],
    x_max=[5, 2],
    dt=5e-3,
    n_steps=[128, 256],
    nu=0.1,
    speed=[1.0, 2.0],
    boundary_condition=bc_neumann_zero,
    ic_kinds=["shock","rarefaction","sine","smooth"],
    n_train=25, n_test=0,
    cfl_safety=1
)
'''

'generate_dataset_burgers(\n    out_dir="generated_1d_burgers",\n    nbx=128,\n    x_min=[-5, -2],\n    x_max=[5, 2],\n    dt=5e-3,\n    n_steps=[128, 256],\n    nu=0.1,\n    speed=[1.0, 2.0],\n    boundary_condition=bc_neumann_zero,\n    ic_kinds=["shock","rarefaction","sine","smooth"],\n    n_train=25, n_test=0,\n    cfl_safety=1\n)\n'

In [12]:
# Create a custom data generation function using Burgers1DLearned
'''
def generate_dataset_with_learned_laplacian(
    out_dir="generated_1d_burgers_learned",
    nbx=128,
    x_min=[-5, -2],
    x_max=[5, 2],
    dt=5e-3,
    n_steps=[128, 256],
    nu=0.1,
    speed=[1.0, 2.0],
    boundary_condition=bc_neumann_zero,
    ic_kinds=["shock", "rarefaction", "sine", "smooth"],
    n_train=25,
    n_test=0,
    cfl_safety=1,
    learn_laplacian=True,
    kernel_size=5,
    device='cpu'
):
    """Generate dataset using the learned Laplacian version"""
    train_dir = os.path.join(out_dir, "train")
    test_dir = os.path.join(out_dir, "test")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    xmins = _tolist(x_min)
    xmaxs = _tolist(x_max)
    stepsLs = _tolist(n_steps)
    speeds = _tolist(speed)

    if len(xmins) != len(xmaxs):
        if len(xmins) == 1:
            xmins = xmins * len(xmaxs)
        elif len(xmaxs) == 1:
            xmaxs = xmaxs * len(xmins)
        else:
            raise ValueError("x_min and x_max must have the same length or be scalars.")
    x_pairs = list(zip(xmins, xmaxs))

    def do_split(N, split_dir, xmin_val, xmax_val, nst_val, spd_val):
        for i in range(N):
            # Create grid
            grid = make_grid(nbx, xmin_val, xmax_val, dt, n_steps=int(nst_val))
            
            # Choose initial condition kind
            kind = None if ic_kinds is None else random.choice(ic_kinds)
            ic_fn, kind_used = make_initial_condition_burgers_fn(grid.x, float(spd_val), kind)
            
            # Create learned simulator
            learned_sim = Burgers1DLearned(
                grid=grid,
                nu=float(nu),
                initial_condition=ic_fn,
                boundary_condition=boundary_condition,
                cfl_safety=cfl_safety,
                learn_laplacian=learn_laplacian,
                kernel_size=kernel_size,
                device=device
            )
            
            # Train the Laplacian network if learning is enabled
            if learn_laplacian:
                print(f"Training Laplacian for sample {i}...")
                training_data = learned_sim.generate_training_data(n_samples=50)
                learned_sim.train_laplacian(training_data=training_data, epochs=500, batch_size=8)
            
            # Run simulation
            U = learned_sim.simulate()
            
            # Create tag and filename
            tag = f"{kind_used}|x=[{xmin_val},{xmax_val}]|T={int(nst_val)}|v={float(spd_val)}|learned_laplacian={learn_laplacian}"
            filename = (
                f"sample_{i:04d}"
                f"_x{float(xmin_val):+.2f}_{float(xmax_val):+.2f}"
                f"_v{float(spd_val):.2f}"
                f"_T{int(nst_val)}"
                f"_learned{learn_laplacian}.npz"
            )
            
            # Save results
            grid.save_npz(
                os.path.join(split_dir, filename),
                U, nu=float(nu), speed=float(spd_val), tag=tag
            )
            
            print(f"Saved {filename}")

    # Generate datasets
    for (xmin, xmax) in x_pairs:
        for nst in stepsLs:
            for spd in speeds:
                do_split(n_train, train_dir, xmin, xmax, nst, spd)
                do_split(n_test, test_dir, xmin, xmax, nst, spd)

# Generate dataset with learned Laplacian
generate_dataset_with_learned_laplacian(
    out_dir="generated_1d_burgers_learned",
    nbx=128,
    x_min=[-5, -2],
    x_max=[5, 2],
    dt=5e-3,
    n_steps=[128, 256],
    nu=0.1,
    speed=[1.0, 2.0],
    boundary_condition=bc_neumann_zero,
    ic_kinds=["shock", "rarefaction", "sine", "smooth"],
    n_train=25,
    n_test=0,
    cfl_safety=1,
    learn_laplacian=True,
    kernel_size=5,
    device='cpu'
)
'''

Training Laplacian for sample 0...
Epoch 0, Loss: 3229.493945
  Learned weights: [-0.0819331   1.3238842  -2.5387058   1.3947688  -0.09834523]
Epoch 100, Loss: 2116.188108
  Learned weights: [ 0.12073012  0.97537774 -2.609134    1.8989409  -0.38438347]
Epoch 200, Loss: 2145.969148
  Learned weights: [ 0.12209132  0.97344726 -2.6053298   1.8948456  -0.38465324]
Epoch 300, Loss: 2927.159836
  Learned weights: [ 0.11909266  0.98022443 -2.6187332   1.9054776  -0.38970265]
Epoch 400, Loss: 2291.699251
  Learned weights: [ 0.12019306  0.96991444 -2.6045008   1.8926274  -0.3827841 ]
Saved sample_0000_x-5.00_+5.00_v1.00_T128_learnedTrue.npz
Training Laplacian for sample 1...
Epoch 0, Loss: 2654.550614
  Learned weights: [-0.0828905  1.3371621 -2.5587018  1.400705  -0.0982867]
Epoch 100, Loss: 1540.574618
  Learned weights: [ 0.08938209  1.049819   -2.6807125   1.951712   -0.40412608]
Epoch 200, Loss: 1908.554018
  Learned weights: [ 0.08776996  1.0481194  -2.6803875   1.9498895  -0.4036921 ]
E

ValueError: Unstable explicit step:
  dt=5.000e-03 > safety*min(dx/|u|, dx^2/(2ν))=4.960e-03
  (dx=3.150e-02, ν=1.000e-01, max|u|=1.000e+00, safety=1.00)

In [ ]:
#Simpler version

# Simple demonstration using Burgers1DLearned for a few simulations
def run_learned_simulations_demo():
    """Run a few simulations with learned Laplacian for demonstration"""
    
    # Parameters for the simulation
    nbx = 128
    x_min, x_max = -5, 5
    dt = 5e-3
    n_steps = 128
    nu = 0.1
    speed = 2.0
    
    # Create grid
    grid = make_grid(nbx, x_min, x_max, dt, n_steps=n_steps)
    
    # Create initial condition
    ic_fn, kind_used = make_initial_condition_burgers_fn(grid.x, speed, "shock")
    
    # Create and train learned simulator
    print("Creating learned Burgers simulator...")
    learned_sim = Burgers1DLearned(
        grid=grid,
        nu=nu,
        initial_condition=ic_fn,
        boundary_condition=bc_neumann_zero,
        cfl_safety=1,
        learn_laplacian=True,
        kernel_size=5,
        device='cpu'
    )
    
    # Train the Laplacian network
    print("Training Laplacian network...")
    training_data = learned_sim.generate_training_data(n_samples=50)
    learned_sim.train_laplacian(training_data=training_data, epochs=500, batch_size=8)
    
    # Run simulation
    print("Running simulation with learned Laplacian...")
    U_learned = learned_sim.simulate()
    
    # For comparison, run with analytical Laplacian
    print("Running simulation with analytical Laplacian...")
    analytical_sim = Burgers1DLearned(
        grid=grid,
        nu=nu,
        initial_condition=ic_fn,
        boundary_condition=bc_neumann_zero,
        cfl_safety=1,
        learn_laplacian=False,  # Use analytical Laplacian
        device='cpu'
    )
    U_analytical = analytical_sim.simulate()
    
    # Plot results for comparison
    print("Plotting results...")
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    grid.plot(U_analytical, title="Analytical Laplacian")
    
    plt.subplot(1, 3, 2)
    grid.plot(U_learned, title="Learned Laplacian")
    
    plt.subplot(1, 3, 3)
    diff = U_learned - U_analytical
    grid.plot(diff, title="Difference", cmap='RdBu')
    
    plt.tight_layout()
    plt.show()
    
    # Print learned weights
    if learned_sim.learn_laplacian and learned_sim.laplacian_net is not None:
        weights = learned_sim.laplacian_net.get_effective_weights()
        print(f"Learned Laplacian weights: {weights}")
    
    return U_analytical, U_learned



In [ ]:
# Run the demonstration
U_analytical, U_learned = run_learned_simulations_demo()

In [ ]:
#samuel's
#visualize_random_sample("generated_1d_burgers/train")

In [ ]:
visualize_random_sample("generated_1d_burgers_learned/train", 
                       title_prefix="Burgers with Learned Laplacian")

In [ ]:
# Generate training data
print("Generating training data...")
training_data = generate_training_data_for_learning(n_samples=50)

# Create simulator with learned Laplacian
grid = make_grid(nbx=128, x_min=-5, x_max=5, dt=5e-3, n_steps=128)
ic_fn, _ = make_initial_condition_burgers(grid.x, speed=2.0, kind="sine")

sim_learned = Burgers1DLearned(grid, nu=0.1, ic_fn, 
                              learn_laplacian=True, kernel_size=5)

# Train the Laplacian operator
print("Training Laplacian operator...")
sim_learned.train_laplacian(training_data, epochs=1000)

# Run simulation with learned operator
print("Running simulation with learned Laplacian...")
U_learned = sim_learned.simulate()

# Compare with analytical
sim_analytical = Burgers1D(grid, nu=0.1, ic_fn)
U_analytical = sim_analytical.simulate()

# Data Comparison

In [ ]:
'''
# Utility function to compare learned vs analytical Laplacian
def compare_laplacians(sim_learned, test_function=None):
    """Compare learned vs analytical Laplacian on test functions"""
    if test_function is None:
        # Default test function: sine wave
        x = sim_learned.grid.x
        test_function = np.sin(2 * np.pi * x / (x.max() - x.min()))
    
    analytical = sim_learned.analytical_laplacian(test_function)
    learned = sim_learned.learned_laplacian(test_function)
    
    error = np.mean((analytical - learned)**2)
    print(f"MSE between learned and analytical Laplacian: {error:.6f}")
    
    # Plot comparison
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(sim_learned.grid.x, analytical, 'b-', label='Analytical', linewidth=2)
    plt.plot(sim_learned.grid.x, learned, 'r--', label='Learned', linewidth=2)
    plt.legend()
    plt.title('Laplacian Comparison')
    plt.xlabel('x')
    plt.ylabel('∇²u')
    
    plt.subplot(1, 2, 2)
    plt.plot(sim_learned.grid.x, analytical - learned, 'g-', label='Error')
    plt.legend()
    plt.title('Error')
    plt.xlabel('x')
    plt.ylabel('Difference')
    
    plt.tight_layout()
    plt.show()
    
    return analytical, learned, error


'''    

In [ ]:
# Generate accurate training data
print("Generating training data...")
training_data = generate_training_data_for_learning(n_samples=200)

# Create simulator with learned Laplacian
grid = make_grid(nbx=128, x_min=-5, x_max=5, dt=5e-3, n_steps=128)
ic_fn, _ = make_initial_condition_burgers(grid.x, speed=2.0, kind="sine")

sim_learned = Burgers1DLearned(grid, nu=0.1, initial_condition=ic_fn, 
                              learn_laplacian=True, kernel_size=5)

# Train the Laplacian operator
print("Training Laplacian operator...")
sim_learned.train_laplacian(training_data, epochs=1000)

# Validate the learned operator
print("Validating learned Laplacian...")
#analytical, learned, error = compare_laplacians(sim_learned)

# Run simulation with learned operator
#print("Running simulation with learned Laplacian...")
#U_learned = sim_learned.simulate()

In [13]:
# Visualize the demo results
if 'U_learned' in locals() and 'U_analytical' in locals():
    grid = make_grid(nbx=128, x_min=-5, x_max=5, dt=5e-3, n_steps=128)
    
    plt.figure(figsize=(15, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(U_analytical, extent=[grid.x.min(), grid.x.max(), grid.t.max(), grid.t.min()],
               cmap='seismic', aspect='auto', origin='upper')
    plt.colorbar(label='u')
    plt.title('Analytical Laplacian')
    plt.xlabel('X'); plt.ylabel('T')
    
    plt.subplot(1, 3, 2)
    plt.imshow(U_learned, extent=[grid.x.min(), grid.x.max(), grid.t.max(), grid.t.min()],
               cmap='seismic', aspect='auto', origin='upper')
    plt.colorbar(label='u')
    plt.title('Learned Laplacian')
    plt.xlabel('X'); plt.ylabel('T')
    
    plt.subplot(1, 3, 3)
    diff = U_learned - U_analytical
    plt.imshow(diff, extent=[grid.x.min(), grid.x.max(), grid.t.max(), grid.t.min()],
               cmap='RdBu', aspect='auto', origin='upper')
    plt.colorbar(label='Difference')
    plt.title('Difference')
    plt.xlabel('X'); plt.ylabel('T')
    
    plt.tight_layout()
    plt.show()
else:
    print("Please run the demo first: U_analytical, U_learned = run_learned_simulations_demo()")

Please run the demo first: U_analytical, U_learned = run_learned_simulations_demo()
